# Algorytm MinMax dla gry kółko i krzyżyk

+ Autor: **Łukasz Staniszewski**
+ Temat: **Implementacja uniwersalnego algorytmu minimax i zastosowanie go do gry "kółko i krzyżyk", a następnie zbadanie jego działania dla różnych maksymalnych głębokości drzewa rozbioru**
+ Zadanie: **[LINK](https://github.com/lukasz-staniszewski/ml-algorithms-scratch/blob/main/minimax-tictactoe/task_pl.pdf)**

## Import bibliotek
+ **random** -> w przypadku algorytmu minimax, gdy ruchy są równie dobre, wybieramy losowy z nich
+ **math** -> dla $+\infty$ i $-\infty$

In [1]:
import random
import math
from typing import List
from __future__ import annotations

## Implementacja MiniMax
+ Zadaniem jest implementacja algorytmu minimax w sposób uniwersalny, tzn. żeby działał dla każdej podanej mu gry.
+ Z założenia gra ta musi posiadać metody:
    + game.is_state_terminal(state) -> sprawdzająca czy stan state w grze game jest uznany za terminalny czy nie;
    + game.get_successors(state) -> zwracająca listę stanów będących następnikami stanu state w grze game;
    + game.grade_state(state) -> oceniająca dany stan albo za pomocą funkcji oceny albo za pomocą heurystycznej funkcji oceny;
    + dodatkowo gra game musi, wywołując metodę minimax, oczekiwać na zwrócenie jej stanu, w którym komputer wykonał ruch.
+ Algorytm zostanie podzielony na dwie funkcje, pierwsza - minimax_make_best_move() - będzie zwracała ruch, który umożliwia zminimalizowanie oceny drogi, która będzie liczona (ocena) za pomocą drugiej funkcji: minimax().
+ Algorytm MiniMax w Python:

In [2]:
def minimax_make_best_move(game: TicTacToe, state: TicTacToeState, depth: int) -> TicTacToeState:
    """Returns move, which maximizes the value of the game state.

    Args:
        game (TicTacToe): game which is being played (here TicTacToe)
        state (TicTacToeState): state of given game
        depth (int): max_depth of searching game tree

    Raises:
        Exception: if state is terminal

    Returns:
        TicTacToeState: next move for oponent which is the best for him
    """
    # game should check whether actual state is terminal before executing this function, but to avoid some situations
    if game.is_state_terminal(state) is True:
       raise Exception("State is terminal.")
    
    else:
        succs = game.get_successors(state)
        possible_scores = []
        
        for succ in succs:
            # next is player, he will maximialize
            score = minimax(game=game, state=succ, depth=depth-1, is_max=True)
            possible_scores.append(score)
        
        # list of best states with length of 1 if there is only one best state
        # random.choice() choose randomly one of successors-states if there are more than one best states 
        
        best_moves = []
        best_score = math.inf
        
        for score, succ in zip(possible_scores, succs):
            if score < best_score: # if there is new best score, clear list and add state on 1st position
                best_moves.clear()
                best_moves.append(succ)
                best_score = score
            
            elif score == best_score:
                best_moves.append(succ)
        
        bestMove = random.choice(best_moves)
        return bestMove

def minimax(game: TicTacToe, state: TicTacToeState, depth: int, is_max: bool) -> int:
    """Scores the game state.

    Args:
        game (TicTacToe): game which is being played (here TicTacToe)
        state (TicTacToeState): state of given game
        depth (int): game tree is being searched to this depth
        is_max (bool): True if player is taking a move, False if oponent is taking a move

    Returns:
        int: value of best move
    """
    if game.is_state_terminal(state) is not False or depth <= 0:
        # grade it by w(s) if s is terminal, otherwise use heuristics
        return game.grade_state(state)
    
    succs = game.get_successors(state)
    
    if is_max:
        # player will maximalize
        best_val = -math.inf
        for succ in succs:
            new_val = minimax(game=game, state=succ, depth=depth-1, is_max=False)
            if new_val > best_val:
                best_val = new_val
        return best_val
    
    else:
        # oponent will minimalize
        best_val = math.inf
        for succ in succs:
            new_val = minimax(game=game, state=succ, depth=depth-1, is_max=True)
            if new_val < best_val:
                best_val = new_val
        return best_val

## Implementacja gry TicTacToe
+ Zadaniem jest zaimplementowanie gry w kółko i krzyżyk, tak aby rozgrywka miała miejsce między graczem a komputerem (którego ruchy będą sterowane minimaxem).
+ Gracz powinien dowiadywać się o aktualnym stanie rozgrywki, a także wprowadzać ruchy za pomocą dowolnego interfejsu (w rozwiązaniu postanowiono posłużyć się standardowym IO).
+ Gra może być zarówno rozpoczęta przez użytkownika (gracza) jak i oponenta (komputer sterowany MiniMaxem).
+ Klasa TicTacToe w Python:

In [3]:
class TicTacToe:
    """Represents TicTactToe game.
    """
    def __init__(self, max_depth: int=10, is_player_starting: bool=True) -> None:
        """Constructor

        Args:
            max_depth (int, optional): max tree depth. 
            Defaults to 10.
            
            is_player_starting (bool, optional): True when player starts game, False if oponent. 
            Defaults to True.
        """
        self.actual_state = TicTacToeState()
        self.create_board()
        self.player1 = 'X'
        self.player2 = 'O'
        self.is_finished = False            
        self.max_depth = max_depth
        self.is_stopped = False             
        self.is_player_starting = is_player_starting

    def check_terminal(self, state: TicTacToeState=None) -> bool:
        """Method checks whether state is terminal.

        Args:
            state (TicTacToeState, optional): checks given state if is passed, otherwise checks game actual state. 
            Defaults to None.

        Returns:
            bool: True if state is terminal, False if not, None if there is no possible move and no-one won 
        """
        # checking bias
        board = self.actual_state.board if state is None else state.board
        if board[0] == board[4] == board[8] != ' ' or board[2] == board[4] == board[6] != ' ':
            if state is None:
                self.is_finished = True
            return True
        
        # checking horizontal and vertical
        for row_col_num in range(3):
            # horizontal
            if board[row_col_num * 3] == board[row_col_num * 3 + 1] == board[row_col_num * 3 + 2] != ' ':
                if state is None:
                    self.is_finished = True
                return True
            # vertical
            if board[row_col_num] == board[row_col_num + 3] == board[row_col_num + 6] != ' ':
                if state is None:
                    self.is_finished = True
                return True
        
        # checking if still is possible move
        for character in board:
            if character == ' ':
                if state is None:
                    self.is_finished = False
                return False
        if state is None:
            self.is_finished = None
        return None

    def create_board(self) -> None:
        """Create board with empty cells.
        """
        self.actual_state.make_empty()

    def __str__(self) -> str:
        """Stringifies current game state.

        Returns:
            str: stringified current game state
        """
        return "\n---|---|---\n".join(["|".join([f" {self.actual_state.board[3*row + col]} "
                                                 for col in range(3)]) for row in range(3)])

    def print_status(self) -> None:
        """Prints information who made move, 
        how board actually looks like and 
        information if game is finished.
        """
        # prints who made move
        print(f"\nPLAYER '{self.actual_state.made_move}' MADE MOVE!\n")
        
        # prints actual board
        print(self)
        
        # prints info who won or when there is a draw
        if self.is_finished is True:
            print(f"\nGAME IS FINISHED! PLAYER '{self.actual_state.made_move}' WON!\n")
        elif self.is_finished is None:
            print(f"\nGAME IS FINISHED! NO-ONE WON!\n")

    def play_game(self) -> None:
        """Main loop playing game. Function creates board and loop is on until game is finished. 
        During loop, player and oponent are making moves for a change. 
        If user writes 'stop' during game, game stops.
        """
        print(self)
        self.create_board()
        
        while True:
            # if player is starting, he takes move if board is empty or if computer made move
            if self.is_player_starting:
                if self.actual_state.made_move == self.player2 or self.actual_state.made_move is None:
                    self.get_player_move()
                # computer makes move only if player made move
                else:
                    self.get_minimax_move()
            
            # if computer is starting, he takes move if board empty or player made move
            else:
                # particular case, when board is empty, we do not want to search game tree so deep
                # because there is no player move
                if self.actual_state.made_move is None:
                    # 5 is hardcoded
                    starting_depth = 5 if self.max_depth > 5 else self.max_depth
                    self.get_minimax_move(starting_depth)
                # if player made move
                elif self.actual_state.made_move == self.player1:
                    self.get_minimax_move()
                # if computer made move
                else:
                    self.get_player_move()
            
            # checks whether player typed "stop" to stop program
            if self.is_stopped:
                return
            
            # checks whether game is finished
            self.check_terminal()
            
            # print actual state of game
            self.print_status()
            
            # breaks infinity loop if game is finished by someones win or draw
            if self.is_finished is not False:
                return

    def get_player_move(self) -> None:
        """Method takes an int from standard input, validates it and changes state of game if is ok.
        If user input 'stop' game is stopped.
        """
        # while user do not give a valid input
        while True:
            move_str = input(f"\nPlayer '{self.player1}' move [0-8]: ")
            
            # if user input 'stop'
            if move_str == "stop":
                print("GAME STOPPED")
                self.is_stopped = True
                return
            
            # if user give index on board
            move = int(move_str)
            if 0 <= move <= 8 and self.actual_state.board[move] == " ":
                self.actual_state.board[move] = self.player1
                self.actual_state.made_move = 'X'     # player = 'X'
                return

    def get_minimax_move(self, max_depth: int=None) -> None:
        """Function makes oponent move by using minimax algorithm.
        
        Args:
            max_depth (int, optional): passed to minimax function if given else game max depth.
            Defaults to None.
        """
        if max_depth is None:
            max_depth = self.max_depth
        self.actual_state = minimax_make_best_move(self, self.actual_state, max_depth)
        self.actual_state.made_move = 'O'      # computer = O

    def is_state_terminal(self, state: TicTacToeState) -> bool:
        """Checks if given state is terminal.

        Args:
            state (TicTacToeState): state to check

        Returns:
            bool: True if state is terminal, False if not, None if no possible move and no-one won
        """
        return self.check_terminal(state)

    def grade_state(self, state: TicTacToeState) -> int:
        """Method grades given state.

        Args:
            state (TicTacToeState): state to grade

        Returns:
            int: grade of state, if is terminal returned is 1/0/-1 else heuristic game
        """
        if self.check_terminal(state) is True:
            if state.made_move == 'X':
                state.rate = 50   # player wins
                return 50
            else:
                state.rate = -50  # oponent wins
                return -50
        
        elif self.check_terminal(state) is None:
            state.rate = 0       # draw
            return 0
        
        # state is not terminal, but depth reached 0 -> use heuristic function
        else:                    
            state.rate = self.heuristic_function(state)
            return self.heuristic_function(state)

    def heuristic_function(self, state: TicTacToeState) -> int:
        """Simple heuristic function for tic-tac-toe from lecture.
        Board of heuristic looks like:
         3 | 2 | 3
        ---|---|---
         2 | 4 | 2
        ---|---|---
         3 | 2 | 3
        
        Function sums up values of this board for 'X' positions and subtracts values for 'O' positions.
        I.E.:
                     O | O |
                    ---|---|---
         For board   X | X |     function returns 4 + 2 - 3 - 2 = 1
                    ---|---|---
                       |   |
        
        Before checking what is sum of values, functions checks whether given state is terminal
        and returns 50 if player wins by doing this move and -50 if player lose by doing this move.
        
        Args:
            state (TicTacToeState): state to count heuristic for

        Returns:
            int: heuristic grade of state
        """
        result_of_move = self.is_state_terminal(state)
        if result_of_move is True and state.made_move == 'X':
            return 50
        elif result_of_move is True and state.made_move == 'O':
            return -50
        
        heuristic_sum = 0
        heuristic_board = [3, 2, 3, 2, 4, 2, 3, 2, 3]
        bad_character = 'O'
        
        for index in range(len(state.board)):
            if state.board[index] == state.made_move:
                heuristic_sum += heuristic_board[index]
            elif state.board[index] == bad_character:
                heuristic_sum -= heuristic_board[index]
        
        return heuristic_sum

    def get_successors(self, state: TicTacToeState) -> List[TicTacToeState]:
        """Returns possible successors of given state.
        For example state:
            state.board = [
            'X', 'O', 'X',
            'X', 'O', 'O',
            ' ', 'X', ' ' ]
            state.rate = None
            state.made_move = 'X'
        Example successors:
            successors[0].board = [                          successors[1].board = [
            'X', 'O', 'X',                                  'X', 'O', 'X',
            'X', 'O', 'O',                                  'X', 'O', 'O',
            'O', 'X', ' ']                                  ' ', 'X', 'O']
            successors[0].rate = None                        successors[1].rate = None
            successors[0].made_move = 'O'                    successors[1].made_move = 'O'

        Args:
            state (TicTacToeState): state to get successors for

        Returns:
            List[TicTacToeState]: list of successors
        """
        successors = []
        
        for index in range(9):
            if state.board[index] == ' ':
                successors.append(state.create_successor(index, self.is_player_starting))
        
        return successors


class TicTacToeState:
    """
    Represents state of TicTacToe game.
    """
    def __init__(self, board: List[List]=None, made_move: bool=None, rate: int=None) -> None:
        """Constructor.

        Args:
            board (List[List], optional): game board. Defaults to None.
            made_move (bool, optional): who made previous move. Defaults to None.
            rate (int, optional): rate of state. Defaults to None.
        """
        self.board = board
        self.rate = rate
        self.made_move = made_move

    def make_empty(self) -> None:
        """Makes game's board full of spaces.
        """
        self.board = [" " for _ in range(9)]

    def create_successor(self, index: int, is_player_starting: bool) -> TicTacToeState:
        """Creates state successor by changing given index, and made_move attribute.
        
        Args:
            index (int): place on board, which will be changed
            is_player_starting (bool): to determine what to do if computer is starting and no-one made move

        Returns:
            TicTacToeState: new successor of given state
        """
        if self.board[index] == ' ':
            if self.made_move is None:     # for first move in game
                if is_player_starting:
                    new_made_move = 'X'
                else:
                    new_made_move = 'O'
            else:
                new_made_move = 'O' if self.made_move == 'X' else 'X'
            new_board = self.board.copy()
            new_board[index] = new_made_move
            return TicTacToeState(new_board, new_made_move)


## Przeprowadzanie gry:
+ Ustawione zostało ziarno dla losowania liczb pseudolosowych (ułatwia sprawdzanie kolejnych przykładowych gier).
+ Należy stworzyć obiekt reprezentujący grę TicTacToe, jako parametr konstruktora podać maksymalną głebokość drzewa rozbioru, dodatkowo podając jako następny parametr konstruktora False, ustawić rozpoczynającego rozgrywkę na oponenta (domyslnie rozpoczyna user).
+ Na końcu należy zawołać funkcję obiektu reprezentującego grę - play_game().

#### Gra rozpoczynana przez użytkownika

In [4]:
random.seed(100)
ttt = TicTacToe(16)
ttt.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   | X 
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   | X |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   | X | O 

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
 X | O |   
---|---|---
   | X | O 

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
 X | O |   
---|---|---
 O | X | O 

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
 X | O | X 
---|---|---
 O | X | O 

GAME IS FINISHED! NO-ONE WON!



#### Gra rozpoczynana przez oponenta

In [5]:
ttt2 = TicTacToe(9, False)
ttt2.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   |   
---|---|---
   | O |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   | O |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
 O | O |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
 O | O | X 

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
 O | O | X 

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
   | O |   
---|---|---
 O | O | X 

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
   | O |   
---|---|---
 O | O | X 

GAME IS FINISHED! PLAYER 'O' WON!



## Działanie dla różnych maksymalnych głębokości drzewa rozbioru
+ Poniżej zostanie zbadane zachowanie algorytmu dla różnych maksymalnych głębokości drzewa rozbioru.
+ Sprawdzenie będzie dokonywane zarówno dla przypadku gdy gracz rozpoczyna rozgrywkę jak i komputer.
+ Przedstawione zostaną przykładowe rozgrywk: gdy gracz gra najlepiej jak może oraz gdy popełni błąd przesądzający o porażce.
+ Badane maksymalne głębokości drzewa rozbioru (max_depth): 8, 5, 2, 0 (dla TicTacToe maksymalna glebokosc mozliwa to wlasnie 8):

#### Dla max_depth = 8

+ Gracz rozpoczyna:

In [6]:
ttt111 = TicTacToe(8)
ttt111.play_game()
ttt112 = TicTacToe(8)
ttt112.play_game()
ttt113 = TicTacToe(8)
ttt113.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | X | O 
---|---|---
   | O |   
---|---|---
 X |   |   

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
 O | O |   
---|---|---
 X |   |   

PLAYER 'X' MADE MOVE!

 X | X | O 
---|---|---
 O | O | X 
---|---|---
 X |   |   

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
 O | O | X 
---|---|---
 X |   | O 

PLAYER 'X' MADE MOVE!

 X | X | O 
---|---|---
 O | O | X 
---|---|---
 X | X | O 

GAME IS FINISHED! NO-ONE WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

   | X |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 

+ Oponent rozpoczyna:

In [27]:
ttt121 = TicTacToe(8, False)
ttt121.play_game()
ttt122 = TicTacToe(8, False)
ttt122.play_game()
ttt123 = TicTacToe(8, False)
ttt123.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   |   
---|---|---
 O |   |   

PLAYER 'X' MADE MOVE!

   |   |   
---|---|---
   | X |   
---|---|---
 O |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   | X | O 
---|---|---
 O |   |   

PLAYER 'X' MADE MOVE!

   | X |   
---|---|---
   | X | O 
---|---|---
 O |   |   

PLAYER 'O' MADE MOVE!

   | X |   
---|---|---
   | X | O 
---|---|---
 O | O |   

PLAYER 'X' MADE MOVE!

   | X |   
---|---|---
   | X | O 
---|---|---
 O | O | X 

PLAYER 'O' MADE MOVE!

 O | X |   
---|---|---
   | X | O 
---|---|---
 O | O | X 

PLAYER 'X' MADE MOVE!

 O | X | X 
---|---|---
   | X | O 
---|---|---
 O | O | X 

PLAYER 'O' MADE MOVE!

 O | X | X 
---|---|---
 O | X | O 
---|---|---
 O | O | X 

GAME IS FINISHED! PLAYER 'O' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   |   
---|---|---
   | O |   

PLA

Jak widać niezależnie czy rozgrywka rozpoczynana jest przez uzytkownika czy oponenta, gra konczy się albo remisem (w przypadku nie popełniania błędnych ruchów przez użytkownika) albo zwycięstwem oponenta.
Parametr max_depth = 8 zapewnia więc, że komputer jest nie do pokonania, algorytm analizuje wtedy każdy możliwy ruch.

#### Dla max_depth = 5:

+ Gracz rozpoczyna: 

In [28]:
ttt211 = TicTacToe(5)
ttt211.play_game()
ttt212 = TicTacToe(5)
ttt212.play_game()
ttt213 = TicTacToe(5)
ttt213.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   | O 

PLAYER 'X' MADE MOVE!

 X |   | X 
---|---|---
   |   |   
---|---|---
   |   | O 

PLAYER 'O' MADE MOVE!

 X |   | X 
---|---|---
   | O |   
---|---|---
   |   | O 

PLAYER 'X' MADE MOVE!

 X | X | X 
---|---|---
   | O |   
---|---|---
   |   | O 

GAME IS FINISHED! PLAYER 'X' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
 O |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
 O |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
 O |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | X | O 
---|---|---
 O |   |   
---|---|---
   |   | X 

PLA

+ Oponent rozpoczyna: 

In [29]:
ttt221 = TicTacToe(5, False)
ttt221.play_game()
ttt222 = TicTacToe(5, False)
ttt222.play_game()
ttt223 = TicTacToe(5, False)
ttt223.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   | O |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | O |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O |   
---|---|---
 O |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | O |   
---|---|---
 O | X |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O |   
---|---|---
 O | X |   
---|---|---
   |   | O 

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
 O | X |   
---|---|---
   |   | O 

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
 O | X |   
---|---|---
 O |   | O 

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
 O | X |   
---|---|---
 O | X | O 

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
 O | X | O 
---|---|---
 O | X | O 

GAME IS FINISHED! NO-ONE WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   |   
---|---|---
 O |   |   

PLAYER 

+ W tym przypadku zachowanie algorytmu jest nieco inne.
+ Gdy grę rozpoczyna użytkownik, oponent popełnia błędy jeśli użytkownik będzie grał bezbłędnie, jednak jeśli użytkownik popełni błąd, oponent jest w stanie wygrać.
+ Gdy grę rozpoczyna oponent, jego zachowanie jest podobne jak dla max_depth = 8, to znaczy, że przy błędach użytkownika, oponent wygrywa rozgrywkę; jednak gdy użytkownik gra bezbłędnie, gra kończy się remisem.

#### Dla max_depth = 2:

+ Gracz rozpoczyna:

In [30]:
ttt311 = TicTacToe(2)
ttt311.play_game()
ttt312 = TicTacToe(2)
ttt312.play_game()
ttt313 = TicTacToe(2)
ttt313.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   | X 
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
   | X |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O |   
---|---|---
 O | X |   

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   | O | X 
---|---|---
 O | X |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   | O | X 
---|---|---
 O | X | O 

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
 X | O | X 
---|---|---
 O | X | O 

GAME IS FINISHED! NO-ONE WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 

+ Oponent rozpoczyna:

In [31]:
ttt321 = TicTacToe(2, False)
ttt321.play_game()
ttt322 = TicTacToe(2, False)
ttt322.play_game()
ttt323 = TicTacToe(2, False)
ttt323.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
 O |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
 O | X |   

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
   | O |   
---|---|---
 O | X |   

GAME IS FINISHED! PLAYER 'O' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
   | O |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   | O 
---|---|---
   | O |   
---|---|---
 X |   |   

PLAYER 'O' MADE MOVE!

 X |   | O 
---|---|---
 O | O |   
---|---|---
 X |   |   

PLA

+ W tym przypadku algorytm radzi sobie lekko lepiej (niż w 5.2) gdy grę rozpoczyna użytkownik, dzięki skuteczności funkcji heurystycznej.

#### Dla max_depth = 0: 

+ Gracz rozpoczyna: 

In [32]:
ttt411 = TicTacToe(0)
ttt411.play_game()
ttt412 = TicTacToe(0)
ttt412.play_game()
ttt413 = TicTacToe(0)
ttt413.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | O |   
---|---|---
   | X |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O |   
---|---|---
   | X | O 
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | O |   
---|---|---
   | X | O 
---|---|---
   |   | X 

GAME IS FINISHED! PLAYER 'X' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

 X | O | X 
---|---|---
   |   |   
---|---|---
   | O |   

PLAYER 'X' MADE MOVE!

 X | O | X 
---|---|---
   | X |   
---|---|---
   | O |   

PLA

+ Oponent rozpoczyna: 

In [33]:
ttt421 = TicTacToe(0, False)
ttt421.play_game()
ttt422 = TicTacToe(0, False)
ttt422.play_game()
ttt423 = TicTacToe(0, False)
ttt423.play_game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
   |   |   
---|---|---
   | O |   

PLAYER 'X' MADE MOVE!

   | X |   
---|---|---
   |   |   
---|---|---
   | O |   

PLAYER 'O' MADE MOVE!

   | X |   
---|---|---
   |   | O 
---|---|---
   | O |   

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
   |   | O 
---|---|---
   | O |   

PLAYER 'O' MADE MOVE!

 X | X |   
---|---|---
 O |   | O 
---|---|---
   | O |   

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
 O | X | O 
---|---|---
   | O |   

PLAYER 'O' MADE MOVE!

 X | X |   
---|---|---
 O | X | O 
---|---|---
   | O | O 

PLAYER 'X' MADE MOVE!

 X | X |   
---|---|---
 O | X | O 
---|---|---
 X | O | O 

PLAYER 'O' MADE MOVE!

 X | X | O 
---|---|---
 O | X | O 
---|---|---
 X | O | O 

GAME IS FINISHED! PLAYER 'O' WON!

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   

PLAYER 'O' MADE MOVE!

   |   |   
---|---|---
 O |   |   
---|---|---
   |   |   

PLA

+ Jak widać w tym przypadku, niezależnie kto rozpoczyna rozgrywkę, algorytm nie robi nic bardziej użytecznego niż wywoływanie funkcji heurystycznej w celu oceniania stanów. Oponent chce minimalizować ocenę, więc będzie wybierał najpierw pozycje, dla których funkcja heurystyczna przyjmuje najmniejsze wartości (środki boków planszy, gdzie przyjmowana jest wartość 2).
+ Wyjątek występuje w momencie kiedy zajmuje on 2 skrajne środki boków, a wolny jest środek planszy; wtedy, tak jak zakłada funkcja heurystyczna, wygrywa grę wykonując ruch na środku planszy; jednak żeby wygrał, należy go sprowokować do tego, w przypadku dobrej gry użytkownika, oponent nie ma szans na zwyciestwo.
+ Jak widać więc po powyższych symulacjach, głębokość poszukiwań drzewa rozbioru ma bardzo duże znaczenie dla poprawnego działania algorytmu MiniMax.